<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Advertools - Check website pages status code

**Tags:** #advertools #website #status #code #check #pages

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Last update:** 2023-08-02 (Created: 2023-08-02)

**Description:** This notebook checks the status code of website pages using Advertools. It is usefull for organizations to quickly check the status of their website pages.

**References:**
- [Advertools Documentation](https://advertools.readthedocs.io/en/latest/)
- [HTTP Status Codes](https://www.restapitutorial.com/httpstatuscodes.html)

## Input

### Import libraries

In [ ]:
import advertools as adv

### Setup variables
- `url`: URL of the website page to check

In [ ]:
url = "https://www.example.com/"

## Model

### Check website page status code

In [ ]:
status_code = adv.check_status_code(url)

## Output

### Display result

In [ ]:
print(f"The status code of {url} is {status_code}.")